In [13]:
import pandas as pd
import random
import time
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver  # 동적크롤링
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [14]:
df = pd.read_csv("Data/제주도식당_전처리.csv", index_col=False)

In [15]:
df.head()

,사업장명,업종구분대분류,업종구분소분류,소재지전체주소,도로명전체주소,lat,lng
0,탑동에이치지,일반음식점,기타,제주특별자치도 제주시 삼도이동 1120-2,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",33.5151128,126.5202001
1,에릭스에스프레소,일반음식점,기타,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",33.5214225,126.8588743
2,일품순두부한림점,일반음식점,한식,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",33.4192601,126.2674304
3,김복남맥주제주아라점,일반음식점,식육(숯불구이),제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",33.4714278,126.5457848
4,봉플라봉뱅,일반음식점,기타,제주특별자치도 제주시 연동 312-57번지 정인하우스,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)",33.4879976,126.4978933


# 현재 없는데이터는 17001~19000까지의 데이터.

In [16]:
df.count()

사업장명       20656
업종구분대분류    20656
업종구분소분류    20653
소재지전체주소    20580
도로명전체주소    20656
lat        20656
lng        20656
dtype: int64

In [17]:
df

,사업장명,업종구분대분류,업종구분소분류,소재지전체주소,도로명전체주소,lat,lng
0,탑동에이치지,일반음식점,기타,제주특별자치도 제주시 삼도이동 1120-2,"제주특별자치도 제주시 무근성안길 16, 1층 (삼도이동)",33.5151128,126.5202001
1,에릭스에스프레소,일반음식점,기타,제주특별자치도 제주시 구좌읍 세화리 1397-6번지,"제주특별자치도 제주시 구좌읍 구좌로 77, 1층",33.5214225,126.8588743
2,일품순두부한림점,일반음식점,한식,제주특별자치도 제주시 한림읍 대림리 1845-2번지,"제주특별자치도 제주시 한림읍 한림상로 237, 1층",33.4192601,126.2674304
3,김복남맥주제주아라점,일반음식점,식육(숯불구이),제주특별자치도 제주시 아라일동 6139,"제주특별자치도 제주시 중앙로 581, 에이동 1층 (아라일동)",33.4714278,126.5457848
4,봉플라봉뱅,일반음식점,기타,제주특별자치도 제주시 연동 312-57번지 정인하우스,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)",33.4879976,126.4978933
...,...,...,...,...,...,...,...
20651,못(MOAT),휴게음식점,커피숍,제주특별자치도 서귀포시 남원읍 남원리 2406-10,"제주특별자치도 서귀포시 남원읍 태위로510번길 47-1, 1, 2층",33.2724596,126.6984028
20652,제주하이루탕후루,휴게음식점,일반조리판매,제주특별자치도 서귀포시 서귀동 271-40,"제주특별자치도 서귀포시 중정로73번길 20, 1층 (서귀동)",33.2498829,126.5639792
20653,연돈,휴게음식점,일반조리판매,제주특별자치도 서귀포시 색달동 2132-2,"제주특별자치도 서귀포시 색달로 10, 2층 (색달동)",33.2588634,126.4061156
20654,가시림 카페,휴게음식점,커피숍,제주특별자치도 서귀포시 표선면 가시리 3275,제주특별자치도 서귀포시 표선면 녹산로5번길 171,33.3610266,126.7516155


### 크롤링에 필요한 컬럼만 뽑아서 저장하기

In [18]:
scrapy_df = df[['사업장명', "도로명전체주소"]]

In [19]:
scrapy_df['도로명전체주소'] = scrapy_df['도로명전체주소'].str.split(',', n=1).str[0]

In [20]:
scrapy_df.head()

,사업장명,도로명전체주소
0,탑동에이치지,제주특별자치도 제주시 무근성안길 16
1,에릭스에스프레소,제주특별자치도 제주시 구좌읍 구좌로 77
2,일품순두부한림점,제주특별자치도 제주시 한림읍 한림상로 237
3,김복남맥주제주아라점,제주특별자치도 제주시 중앙로 581
4,봉플라봉뱅,제주특별자치도 제주시 문송1길 6-1


In [21]:
# 크롬 드라이버 실행
# chrome_options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [22]:
url = f"https://map.naver.com/v5/search/"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
driver.get(url)

In [23]:
# for i in range(17001, 17011):
#     url = f"https://map.naver.com/v5/search/{scrapy_df['사업장명'][i]} {scrapy_df['도로명전체주소'][i]}"
# # driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[1]/div/div[4]/a/span[2]/svg').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭

In [24]:
# 리뷰를 추출해주는 함수
def extract_review(listsCount: int):
    time.sleep(random.uniform(2, 2.5))
    rev = []  # 추출한 리뷰 저장
    for i in range(1, 11): 
        try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
            if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                print(f"1~10 try (if) 사진ㅇreview {i}")
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                print(f"1~10 find (if) 사진ㅇreview{i}")
                
                
                time.sleep(0.1)
                comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
                print(f"1~10 get (if) 사진ㅇreview{1}")
                

                rev.append(comment)
            else:
                print(f"1~10 try (else) 사진ㅇreview{i}")
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                print(f"1~10 find (else) 사진ㅇreview{i}")
                time.sleep(0.1)
                comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
                print(f"1~10 get (else) 사진ㅇreview{i}")

                rev.append(comment)
        except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
            try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                    print(f"1~10 try (if) 사진x review{i}")
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                    print(f"1~10 find (if) 사진x review{i}")
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                    print(f"1~10 get (if) 사진x review{i}")
                    
                    rev.append(comment)
                else:
                    print(f"1~10 try (else) 사진x review{i}")
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                    print(f"1~10 find (else) 사진x review{i}")
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                    print(f"1~10 get (else) 사진x review{i}")
                    
                    rev.append(comment)
            except:
                rev.append("")

    # 더 많은 리뷰가 있는지 확인하고, 있을 경우에는 추가 리뷰를 가져오기 위해 "더보기" 버튼 클릭
    try:
        print("더보기 (11~20)")
        if listsCount == 1:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
            print(f"더보기 클릭!if")
        else:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
            print(f"더보기 클릭!else")
        time.sleep(0.5)
        for i in range(11, 21):
            try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
                if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨
                    print(f"11~20 try (if) 더보기 사진ㅇreview {i}")
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                    print(f"11~20 find (if) 더보기 사진ㅇreview {i}")
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
                    print(f"11~20 get (if) 더보기 사진ㅇreview {i}")

                    rev.append(comment)
                else:
                    print(f"11~20 try (else) 더보기 사진ㅇreview {i}")
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                    print(f"11~20 find (else) 더보기 사진ㅇreview {i}")
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
                    print(f"11~20 get (else) 더보기 사진ㅇreview {i}")

                    rev.append(comment)
            except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
                try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                    if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                        print(f"11~20 try (if) 더보기 사진x review {i}")
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                        print(f"11~20 find (if) 더보기 사진x review {i}")
                        time.sleep(0.1)
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        print(f"11~20 get (if) 더보기 사진x review {i}")
                        
                        rev.append(comment)
                    else:
                        print(f"11~20 try (else) 더보기 사진x review {i}")
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                        print(f"11~20 find (else) 더보기 사진x review {i}")
                        time.sleep(0.1)
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        print(f"11~20 get (else) 더보기 사진x review {i}")
                        
                        rev.append(comment)
                except:
                    rev.append("")
    
    except:
        print(f'더 이상 리뷰가 없음{i}')
    
    try:
        print("더보기 (21~30)")
        if listsCount == 1:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)
            print(f"더보기 클릭!if")
        else:
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a').send_keys(Keys.ENTER)   
            print(f"더보기 클릭!else")
        time.sleep(0.5)
        for i in range(21, 31):
            try:  # 사진 있는 후기는 div 3번째에 텍스트 위치
                if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                    print(f"21~30 try (if) 더보기 사진ㅇreview {i}")
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                    print(f"21~30 find (if) 더보기 사진ㅇreview {i}")
                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
                    print(f"21~30 get (if) 더보기 사진ㅇreview {i}")

                    rev.append(comment)
                else:
                    print(f"21~30 try (else) 더보기 사진ㅇreview {i}")
                    driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a').send_keys(Keys.ENTER) # 텍스트 전체 볼 수 있게 클릭
                    print(f"21~30 find (else) 더보기 사진ㅇreview {i}")

                    time.sleep(0.1)
                    comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[3]/a/span').text  # 리뷰
                    print(f"21~30 get (else) 더보기 사진ㅇreview {i}")
                    rev.append(comment)
            except: # 사진 없는 후기는 div 2번째에 텍스트가 위치
                try: # 리뷰에 글이 없는경우에 XPATH가 달라짐으로 try - except 추가
                    if listsCount == 1: # 리뷰버튼 갯수가 1개면 태그가 6331이아닌 6231로됨 
                        print(f"21~30 try (if) 더보기 사진x review {i}")
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                        print(f"21~30 find (if) 더보기 사진x review {i}")
                        time.sleep(0.1)
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        print(f"21~30 get (if) 더보기 사진x review {i}")
                        
                        rev.append(comment)
                    else:
                        print(f"21~30 try (else) 더보기 사진x review {i}")
                        driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a').send_keys(Keys.ENTER)
                        print(f"21~30 find (else) 더보기 사진x review {i}")
                        time.sleep(0.1)
                        comment = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li['+str(i)+']/div[2]/a/span').text  # 리뷰
                        print(f"21~30 get (else) 더보기 사진x review {i}")
                        
                        rev.append(comment)

                except:
                    rev.append("")

    except:
        print('더 이상 리뷰가 없음')

    print("========================================================================================================")    
    
    return rev




In [25]:
count = 0 # 성공한 크롤링 카운트
failedCount = 0 # 실패한 크롤링 카운트

crawlingReviewData = [] # 식당 리뷰를 저장할 리스트
crawlingImageData = [] # 식당 이미지를 저장할 리스트
crawlingCategoryData = [] # 식당 업종분류를 저장할 리스트
crawlingVisitCountData = [] # 식당 방문자리뷰 수를 저장할 리스트
crawlingBlogCountData = [] # 식당 블로그리뷰 수를 저장할 리스트
crawlingRatingData = [] # 식당 평점을 저장할 리스트
errorResult = ""

result_df = pd.DataFrame()
# headers = {'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}

# range(len(scrapy_df))
# range(1, 2)
# for i in range(9001, 10001):
for i in range(17001, 17011):
    url = f"https://map.naver.com/v5/search/{scrapy_df['사업장명'][i]} {scrapy_df['도로명전체주소'][i]}"
    # url = f"https://map.naver.com/v5/search/걸작떡볶이제주노형점 제주특별자치도 제주시 노형9길 7"
    # url = f"https://map.naver.com/v5/search/신촌도시락 제주 제주시 조천읍 신북로 88"
    # url = f"https://map.naver.com/v5/search/우본가해장국 제주특별자치도 제주시 건주로 72"
    
    driver.get(url)
    time.sleep(random.uniform(2.0, 2.5))
    try: 
        driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]')) # iframe 이동
        time.sleep(random.uniform(1.5, 2.0))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 이미지 3장 뽑아서 저장하기
        images = []  # 이미지 링크를 저장할 리스트
        for i in range(1, 4):
            image_element = soup.select_one(f"#ibu_{i}")  # ibu_1 ~ 3 이런식으로 이미지의 id가 갯수별로 형성됨
            if not image_element:
                continue  # 이미지 요소가 없으면 for 루프 종료
            style = image_element["style"]
            background_image = style.split("url(")[1].split(")")[0]
            images.append(background_image)  # 이미지 링크를 리스트에 추가

        # 음식 카테고리 뽑기 (버튼이라서 CSS_SELECTOR를 써야함)
        foodCategory = driver.find_element(By.CSS_SELECTOR, '#_title > div > span.DJJvD').text


        # 리뷰 탭을 선택하기 전에 버튼 갯수 파악하기 (별점 4.7, 방문자리뷰 100, 블로그리뷰 120) 이런식으로 된곳
        lists = soup.select('.PXMot')  # 버튼들의 class = PXMot
        errorResult = "검색 결과 없음"
        if len(lists) == 3: # (별점이 있는 경우) 별점/방문자리뷰/블로그리뷰 순일때 방문자리뷰는 두번째에 위치=span[2]
            # 별점 수, 리뷰 수 긁어오기
            ratingText = soup.select_one('.PXMot.LXIwF').text # 별점의 class = .PXMot.LXIwF
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(3) > a').text

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').send_keys(Keys.ENTER) # 방문자 리뷰
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1)
        elif len(lists) == 2: # (별점이 없는 경우) 방문자리뷰/블로그리뷰 순일때 방문자리뷰는 첫번째에 위치=span[1]
            # 리뷰 수 긁어오기                                 
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(1) > a').text
            blogText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span:nth-child(2) > a').text
            ratingText = "x"

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1)
        elif len(lists) == 1: # 블로그 리뷰가 없는 경우 
            visitText = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span > a').text
            blogText = "블로그리뷰 0"
            ratingText = "x"

            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span/a').send_keys(Keys.ENTER) # 방문자 리뷰 클릭
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
            time.sleep(1) 
        else:
            errorResult = "크롤링 불가"
                                        
        
        # time.sleep(random.uniform(0.8, 1.0))
        # review = extract_review(int(re.sub(r'\D', '', visitText)))  # 정규표현식을 사용하여 숫자만 추출
        review = extract_review(len(lists))

        crawlingReviewData.append(review)
        crawlingImageData.append(images)
        crawlingCategoryData.append(foodCategory)
        crawlingRatingData.append(ratingText.replace("별점", ""))
        crawlingVisitCountData.append(int(re.sub(r'\D', '', visitText)))  # 정규표현식을 사용하여 숫자만 추출
        crawlingBlogCountData.append(int(re.sub(r'\D', '', blogText)))
        count += 1
        print("----------------------------------")
        print(f'크롤링 성공! 성공: {count}, 실패: {failedCount}')
        ## 별점이 있고 없고에 따라 공백 데이터 추가해주는거 필요
    except:
        crawlingReviewData.append('x')
        crawlingImageData.append('x')
        crawlingCategoryData.append('x')
        crawlingRatingData.append('x')
        crawlingVisitCountData.append("x")
        crawlingBlogCountData.append("x")
        failedCount += 1
        print("----------------------------------")
        print(f'크롤링 실패! 성공: {count}, 실패: {failedCount}, 이유: {errorResult}')

    result_df = pd.DataFrame({'foodCategory': crawlingCategoryData, 'images': crawlingImageData, 'rating': crawlingRatingData, 'visitReviewCount': crawlingVisitCountData, 'blogReviewCount': crawlingBlogCountData,'review': crawlingReviewData})


----------------------------------
크롤링 실패! 성공: 0, 실패: 1, 이유: 검색 결과 없음
1~10 try (else) 사진ㅇreview1
1~10 try (else) 사진x review1
1~10 try (else) 사진ㅇreview2
1~10 try (else) 사진x review2
1~10 try (else) 사진ㅇreview3
1~10 try (else) 사진x review3
1~10 try (else) 사진ㅇreview4
1~10 try (else) 사진x review4
1~10 try (else) 사진ㅇreview5
1~10 try (else) 사진x review5
1~10 try (else) 사진ㅇreview6
1~10 try (else) 사진x review6
1~10 try (else) 사진ㅇreview7
1~10 try (else) 사진x review7
1~10 try (else) 사진ㅇreview8
1~10 try (else) 사진x review8
1~10 try (else) 사진ㅇreview9
1~10 try (else) 사진x review9
1~10 try (else) 사진ㅇreview10
1~10 try (else) 사진x review10
더보기 (11~20)
더보기 클릭!else
11~20 try (else) 더보기 사진ㅇreview 11
11~20 try (else) 더보기 사진x review 11
11~20 try (else) 더보기 사진ㅇreview 12
11~20 try (else) 더보기 사진x review 12
11~20 try (else) 더보기 사진ㅇreview 13
11~20 try (else) 더보기 사진x review 13
11~20 try (else) 더보기 사진ㅇreview 14
11~20 try (else) 더보기 사진x review 14
11~20 try (else) 더보기 사진ㅇreview 15
11~20 try (else) 더보기 사진x review 15
11~20 try

In [34]:
result_df

,foodCategory,images,rating,visitReviewCount,blogReviewCount,review
0,x,x,x,x,x,x
1,갤러리카페,"[""https://search.pstatic.net/common/?autoRotat...",4.23,66,11,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
2,"카페,디저트","[""https://search.pstatic.net/common/?autoRotat...",4.23,66,11,"[, , , , , , , , , ]"
3,x,x,x,x,x,x
4,라면,"[""https://search.pstatic.net/common/?autoRotat...",4.45,805,824,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
5,x,x,x,x,x,x
6,x,x,x,x,x,x
7,x,x,x,x,x,x
8,라면,"[""https://search.pstatic.net/common/?autoRotat...",4.79,12159,2572,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
9,"카페,디저트","[""https://search.pstatic.net/common/?autoRotat...",4.53,241,18,"[, , , , , , , , , , , , , , , , , , , , , , ,..."


In [32]:
# 최신순 버튼 클릭 못하네
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a').send_keys(Keys.ENTER) # 최신순으로 클릭
                          #    //*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[1]/div[2]/div[1]/span[2]/a 
                                                                    #  ㄴ기존 코드랑 달라진 부분

In [58]:
###  본인작성 리뷰만 가져오자. 자동작성 리뷰는 긍정뿐이 없음.
# 2번리뷰 (사진 X, 본인작성 리뷰, 상세한 리뷰) - 상세한 리뷰 ( 이용방법, 대기시간, 목적, 동행 등을 입력한 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[3]/a/span').text 
# 6번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 ) - 기존리뷰
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[6]/div/div[2]/a/span').text
# 8번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[8]/div/div[2]/a/span').text
# 9번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[9]/div/div[2]/a/span').text
# 10번리뷰 (사진 X, 본인작성 리뷰, 기존 리뷰 )



# 3번 리뷰 (사진 O, 본인작성 리뷰)
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[3]/div/div[4]/a/span').text
# 4번 리뷰 (사진 O, 본인작성 리뷰)
# print(4)
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[4]/div/div[4]/a/span').text


# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text
# driver.find_element(By.XPATH, '').text



'케이크 종류가 많고 다 평타 이상으로 맛있음 층고가 엄청 높아서 분위기 있음 위치가 시청치고 시끄러운 편도 아니어서 얘기 나누기도 좋음'

In [57]:
# 2번리뷰 (사진 X, 본인작성 리뷰)
driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/ul/li[2]/div/div[3]/a/span').text 

'꽃차 추천'